In [1]:
import os
import torch
from random import randint
import numpy as np
import gym
from gym.spaces import Box, Discrete, MultiBinary
import cv2
import math
import matplotlib.pyplot as plt
import time
import pandas as pd
from tqdm.notebook import tqdm
from multiprocessing import Pool
import time
# import scipy
# import scipy.stats
from my_function import save_variable, load_variable, cor_to_ind, ind_to_cor, cor_to_nor, nor_to_cor, lat_to_cor, a_star_search
from my_ppo import PPO, Memory, ActorCritic, Critic, Actor
from my_env import MyEnv
log_interval = 1000  # print avg reward in the interval
max_timesteps = 300  # max timesteps in one episode
update_timestep = 300*30  # update policy every n timesteps 800*24 (max memory size)
action_var = [0.2, 0.2]  # constant variance for action distribution (Multivariate Normal)
K_epochs = 40  # update policy for K epochs
eps_clip = 0.2  # clip parameter for PPO
gamma = 0.99  # discount factor
lr = 0.00005  # parameters for Adam optimizer
betas = (0.9, 0.999)
random_seed = 3
train_env = MyEnv(train = True)
state_dim = 32
action_dim = train_env.action_space.shape[0]

if random_seed:
    print("Random Seed: {}".format(random_seed))
    torch.manual_seed(random_seed)
    np.random.seed(random_seed)
    
train_memory = Memory()
ppo = PPO(state_dim, action_dim, action_var, lr, betas, gamma, K_epochs, eps_clip)
# ppo = load_variable('ppo_best') # 
running_reward = 0
avg_length = 0
time_step = 0
update_time = 0

/root/miniconda3/lib/python3.8/site-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Random Seed: 3


In [ ]:
train_df = load_variable('train_df')
test_df = load_variable('test_df')
columns = ['Episode', 'AvgLen', 'AvgR', 'AvgV', 'Speedvar','Succ_Rate','Test_time', 'Train_time']
test_result = pd.DataFrame(columns=columns)
T1= time.perf_counter()

save_reward = []
save_speed = []
save_episode = []
save_length = []
save_speedvar = []
last_reward = 0

for i_episode in range(0, len(train_df)):
    state_1, state_2 = train_env.reset()
    train_temp_rewards = 0
    for t in range(max_timesteps):
        time_step += 1
        # Running policy_old:
        action = ppo.select_action(state_1, state_2, train_memory)
        state_1, state_2, reward, done= train_env.step(action)
        train_temp_rewards += reward
        # Saving reward and is_terminals:
        train_memory.rewards.append(reward)
        train_memory.is_terminals.append(done)
        # if time_step % update_timestep == 0:
        #     flag = True
        # if flag:
        #     if t == max_timesteps-1 or done:
        #         ppo.update(train_memory)
        #         train_memory.clear_memory()
        #         time_step = 0
        #         flag = False
        if time_step % update_timestep == 0:
            ppo.update(train_memory)
            train_memory.clear_memory()
            time_step = 0
        if done:
            break        
    
    if (i_episode+1) % log_interval == 0:
        path = 'ppoF/ppo' + str(i_episode+1)
        print('Episode: {} \t AvgR: {:.2f}'.format(i_episode+1, train_temp_rewards))
        save_variable(ppo, path)
        # T2 = time.perf_counter()
        # test_env = MyEnv(train = False)
        # test_memory = Memory()
        # speeds = []
        # lengths = []
        # rewards = []
        # test_start_time = time.perf_counter()
        # succes_num = 0
        # for j in range(0, 200):
        #     state_1, state_2 = test_env.reset()
        #     reward = 0
        #     for j_t in range(500):
        #         action = ppo.select_action(state_1, state_2, test_memory) # 在计算action时，把state, action, logprobs存入 memory
        #         state_1, state_2, reward, done= test_env.step(action)
        #         reward += reward
        #         speed = np.linalg.norm(state_1[2:4]*5, ord = 2)
        #         speeds.append(speed)
        #         if done:
        #             succes_num += 1
        #             break
        #     test_memory.clear_memory()
        #     lengths.append(j_t+1)
        #     rewards.append(reward)
        # test_end_time = time.perf_counter()
        # avg_length = np.mean(lengths)
        # avg_reward = np.mean(rewards)
        # avg_speed = np.mean(speeds)
        # var_speed = np.var(speeds)
        # success_rate = succes_num/(len(test_df)-300)
        # test_time = test_end_time - test_start_time   
        # train_time = T2-T1
        # new_row = [i_episode+1, avg_length, avg_reward, avg_speed, var_speed,success_rate, test_time, train_time]
        # test_result.loc[len(test_result)] = new_row
        # T1 = time.perf_counter()
        # print('Episode {} \t AvgLen: {:.2f} \t AvgR: {:.2f} \t AvgV: {:.2f} \t Var_V:{:.2f} \t Succ_Rate: {:.2f} \t Test_T: {:.2f}s \t Train_T: {:.2f}s '.format(i_episode+1, avg_length, 
        #             avg_reward, avg_speed, var_speed, success_rate, test_time, train_time))

Episode: 1000 	 AvgR: 137.55
Episode: 2000 	 AvgR: 243.68
Episode: 3000 	 AvgR: 131.50
Episode: 4000 	 AvgR: 406.87
Episode: 5000 	 AvgR: -22.77
Episode: 6000 	 AvgR: 231.17
Episode: 7000 	 AvgR: 366.70
Episode: 8000 	 AvgR: 391.22
Episode: 9000 	 AvgR: 484.34
Episode: 10000 	 AvgR: 504.02
Episode: 11000 	 AvgR: 162.79
Episode: 12000 	 AvgR: 423.17
Episode: 13000 	 AvgR: 241.85
Episode: 14000 	 AvgR: 147.90
Episode: 15000 	 AvgR: 135.19
Episode: 16000 	 AvgR: 520.55
Episode: 17000 	 AvgR: 516.25
